In [1]:
# gonna give pycaret a try
import pandas as pd
from pycaret.regression import RegressionExperiment

In [2]:
DATA_TRAIN = '.data/train.csv'
DATA_TEST_X = '.data/test.csv'
DATA_TEST_Y = '.data/revealed_targets.csv'

KAGGLE_DATA_TRAIN = '/kaggle/input/optiver-trading-at-the-close/train.csv'
KAGGLE_DATA_TEST_X = '/kaggle/input/optiver-trading-at-the-close/example_test_files/test.csv'
KAGGLE_DATA_TEST_Y = '/kaggle/input/optiver-trading-at-the-close/example_test_files/revealed_targets.csv'

DROPS = ['index', 'time_id', 'currently_scored', 'time_id_x', 'time_id_y', 'revealed_date_id', 'revealed_time_id']
SORTS = ['date_id', 'seconds_in_bucket', 'stock_id']
INDEX = 'row_id'

def load_vars(testing:bool=False) -> pd.DataFrame:

    def read_data(train, test_x, test_y):
        if testing:
            data = pd.merge(*[pd.read_csv(path) for path in [test_x, test_y]], on=SORTS) # https://stackoverflow.com/a/32041277/3178898
            ycol = 'revealed_target'
        else:
            data = pd.read_csv(train, index_col=INDEX)
            ycol = 'target'
        return data, ycol
    
    try: # tired of switching local/kaggle setup
        data, ycol = read_data(DATA_TRAIN, DATA_TEST_X, DATA_TEST_Y)
    except FileNotFoundError:
        data, ycol = read_data(KAGGLE_DATA_TRAIN, KAGGLE_DATA_TEST_X, KAGGLE_DATA_TEST_Y)

    data = data.dropna(subset=[ycol]) # some targets are null
    return data

In [3]:
exp = RegressionExperiment()
exp.setup(load_vars(), target='target', session_id=489, use_gpu=True, verbose=False)

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more 

In [5]:
best = exp.compare_models() # blast-off modifier

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,15:09:45
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,6.3755,88.4659,9.4055,0.0113,1.4876,1.2650,2.6740
ridge,Ridge Regression,6.3874,88.8720,9.4271,0.0068,1.6474,1.1077,1.4500
br,Bayesian Ridge,6.4091,89.0054,9.4342,0.0053,1.3188,1.5696,2.8380
lasso,Lasso Regression,6.4070,89.4138,9.4558,0.0007,1.7908,1.0340,2.3380
en,Elastic Net,6.4070,89.4137,9.4558,0.0007,1.7906,1.0341,2.4420
llar,Lasso Least Angle Regression,6.4070,89.4138,9.4558,0.0007,1.7908,1.0340,1.4890
huber,Huber Regressor,6.4070,89.4158,9.4559,0.0007,1.7891,1.0385,10.3750
omp,Orthogonal Matching Pursuit,6.4106,89.4810,9.4593,-0.0000,1.8217,1.0076,1.4560
knn,K Neighbors Regressor,6.7711,93.6353,9.6764,-0.0464,1.0438,3.1561,12.3920
lar,Least Angle Regression,7.2338,146.1901,11.0091,-0.6300,1.4168,2.2720,1.4550


Processing:   0%|          | 0/85 [00:00<?, ?it/s]

KeyboardInterrupt: 